In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from bs4 import BeautifulSoup
import geocoder
import requests
import folium
import geopy.distance
import json
from folium.plugins import MousePosition
from folium.plugins import Draw


load_dotenv()


def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, austin = connectCollection('companies','companies')
db, start = connectCollection('companies','startups')
db, filtered = connectCollection('companies','filtered')
db, starbucks = connectCollection('companies','starbucks')
db, schools = connectCollection('companies','schools')
db, night = connectCollection('companies','night')

In [2]:
# encontramos todas las empresas con oficinas en Boston, MA
# estas empresas deben seguir existiendo (deadpooled==None)
# debemos saber el año en el que fueron fundadas (founded_year==exists)


austin_comp = list(austin.find({"$and":[{"offices.city":"Boston"},{"deadpooled_year":None},
                                        {"founded_year":{"$exists":True}}]}))



In [3]:
# creamos una nueva coleccion con los datos filtrados
# filtramos por el año en el que fueron fundadas (no mas de 10 años)
# separamos las empresas en sus distintas oficinas


for y in austin_comp:
    if y["founded_year"]==None or y["founded_year"]>2009:
        pass
    else:
        for of in range(len(y["offices"])):
            if y["offices"][of]["city"]=="Boston":
                db.filtered.insert_one(
                {"name":y["name"],"homepage_url":y["homepage_url"],"category_code":y["category_code"],
                "number_of_employees":y["number_of_employees"],"founded_year":y["founded_year"],
                "total_money_raised":y["total_money_raised"],"offices":y["offices"][of]})
                

In [4]:
def getLocation(_lng,_lat):
    longitude = _lng
    latitude = _lat
    try:
        loc = {
            'type':'Point',
            'coordinates':[float(longitude), float(latitude)]
        }
        return loc
    except:
        pass

In [5]:
# funcion para geolocalizar direcciones

def place_request(direction):
    if not os.getenv("google"):
        raise ValueError("No API token!")
    else:
        g = geocoder.google(direction,key=os.getenv("google"))
        return g.json
    

In [6]:
filter_list = filtered.find()
for i in filter_list:
    lng = i["offices"]["longitude"]
    lat = i["offices"]["latitude"]
    geocode = {"$set": {'location':getLocation(lng,lat)}}
    filtered.update_one(i,geocode)

In [7]:
# de angel.co sacamos un csv filtrado de startups en Boston que han ganado entre 2 y 100 M
# https://angel.co/
# subimos ese csv a mongo db compass desde la terminal
# mongoimport --db companies --collection companies --file startups.csv

startups = list(start.find())
startups = [startups[s] for s in range(len(startups)-1) if int(startups[s]["Total Raised"])>1000000]

# obtenemos la direccion de todas las startups en Boston (nombre y estado)

for startup in startups[:-1]:
    #x = place_request(startup["Name"]+",MA")
    try:
        lat = x["raw"]["geometry"]["location"]["lat"]
        lng = x["raw"]["geometry"]["location"]["lng"]
        geocode = {"$set": {'location':getLocation(lng,lat)}}
        start.update_one(startup,geocode)
    except:
        pass


In [8]:
# https://www.kaggle.com/crawford/boston-public-schools
# CSV con los colegios publicos en Boston de los que sacamos la localizacion

school = list(schools.find())
for s in school:
    lng = s["\ufeffX"]
    lat = s["Y"]
    geocode = {"$set": {'location':getLocation(lng,lat)}}
    schools.update_one(s,geocode)


In [9]:
def request_json(url):
    res = requests.get(url).json()
    return res["results"]

def getAdress(results):
    return results["formatted_address"]

def getPosition(results):
    lng = results["geometry"]["location"]["lng"]
    lat = results["geometry"]["location"]["lat"]
    return lng,lat
    

In [10]:
### starbucks search
url_starbucks = (f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=starbucks+in+Boston&key={os.getenv('google')}")
#starbucks_info = request_json(url_starbucks)
for s in starbucks_info:
    lng,lat = getPosition(s)
    db.starbucks.insert_one({'location':getLocation(lng,lat),'adress':getAdress(s)})

### night search
url_night = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=night+bars+Boston&key={os.getenv('google')}"
#night_info = request_json(url_night) 
for n in night_info:
    lng,lat = getPosition(n)
    db.night.insert_one({'location':getLocation(lng,lat),'adress':getAdress(n)})    
    

In [11]:
# we set the initial location as the center of Boston, from googlemaps

#city = place_request("boston,MA")
#airport = place_request("boston airport")
#air_coord = airport.get('lat'),airport.get('lng')

In [26]:
filtered_2 = filtered.find({"location.coordinates":{"$exists":True}})
start_2 = start.find({"location.coordinates":{"$exists":True}})
schools_2 = schools.find({"location.coordinates":{"$exists":True}})
starbucks_2 = starbucks.find({"location.coordinates":{"$exists":True}})
night_2 = night.find({"location.coordinates":{"$exists":True}})

In [13]:
m = folium.Map([city.get("lat"),city.get("lng")], zoom_start=15,tiles='cartodbpositron')
     
school_group = folium.FeatureGroup(name="Schools").add_to(m)
start_group = folium.FeatureGroup(name="Startups").add_to(m)
companies_group = folium.FeatureGroup(name="Companies").add_to(m)
starbucks_group = folium.FeatureGroup(name="Starbucks").add_to(m) 
night_group = folium.FeatureGroup(name="Night bars").add_to(m)
plane_group = folium.FeatureGroup(name="Airport").add_to(m)
    
    
for c in list(filtered_2):
    coord = c["location"]["coordinates"][::-1]
    icon=folium.Icon(color='black',prefix='companies')
    companies_group.add_child(folium.Marker(coord,popup="Old company "+str(coord),icon=icon))   

for school in list(schools_2):
    coord = school["location"]["coordinates"][::-1]
    icon=folium.Icon(color='darkblue', icon='home', icon_color="red", prefix='School')
    school_group.add_child(folium.Marker(coord,popup="School "+str(coord),icon=icon))

for start_loc in list(start_2):
    coord = start_loc["location"]["coordinates"][::-1]
    icon=folium.Icon(color='orange', prefix='startup')
    start_group.add_child(folium.Marker(coord,popup="Startup "+str(coord),icon=icon))

for star in list(starbucks_2):
    coord = star["location"]["coordinates"][::-1]
    icon=folium.Icon(color='darkgreen', prefix='starbucks')
    starbucks_group.add_child(folium.Marker(coord,popup="Starbucks "+str(coord),icon=icon))
    
for club in list(night_2):
    coord = club["location"]["coordinates"][::-1]
    icon=folium.Icon(color='purple', prefix='Night bar')
    night_group.add_child(folium.Marker(coord,popup="Night bar "+str(coord),icon=icon))

icon=folium.Icon(icon='cloud',color='lightblue', prefix='Airport')
plane_group.add_child(folium.Marker(air_coord,popup="Airport "+str(air_coord),icon=icon))
    
folium.LayerControl(collapsed=True).add_to(m)
folium.LatLngPopup().add_to(m)

draw = Draw(export=True)



formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
MousePosition(
    position='topright',
    separator=' | ',
    lng_first=True,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m)



draw.add_to(m)


In [151]:
schools_2 = schools.find({"location":{"$geoWithin":{"$centerSphere":[[-71.05980241470984,42.36022870301706 ],0.0001756461077225538]}}})
filtered_2 = filtered.find({"location":{"$geoWithin":{"$centerSphere":[[-71.05980241470984,42.36022870301706 ],0.0001756461077225538]}}})
start_2 = start.find({"location":{"$geoWithin":{"$centerSphere":[[-71.05980241470984,42.36022870301706 ],0.0001756461077225538]}}})
starbucks_2 = starbucks.find({"location":{"$geoWithin":{"$centerSphere":[[-71.05980241470984,42.36022870301706 ],0.0001756461077225538]}}})
night_2 = night.find({"location":{"$geoWithin":{"$centerSphere":[[-71.05980241470984,42.36022870301706 ],0.0001756461077225538]}}})





In [127]:
m2 = folium.Map([city.get("lat"),city.get("lng")], zoom_start=15,tiles='cartodbpositron')
     
school_group = folium.FeatureGroup(name="Schools").add_to(m2)
start_group = folium.FeatureGroup(name="Startups").add_to(m2)
companies_group = folium.FeatureGroup(name="Companies").add_to(m2)
starbucks_group = folium.FeatureGroup(name="Starbucks").add_to(m2) 
night_group = folium.FeatureGroup(name="Night bars").add_to(m2)
plane_group = folium.FeatureGroup(name="Airport").add_to(m2)
    
    
for c in list(filtered_2):
    coord = c["location"]["coordinates"][::-1]
    icon=folium.Icon(color='black',prefix='companies')
    companies_group.add_child(folium.Marker(coord,popup="Old company "+str(coord),icon=icon))   

for school in list(schools_2):
    coord = school["location"]["coordinates"][::-1]
    icon=folium.Icon(color='darkblue', icon='home', icon_color="red", prefix='School')
    school_group.add_child(folium.Marker(coord,popup="School "+str(coord),icon=icon))

for start_loc in list(start_2):
    coord = start_loc["location"]["coordinates"][::-1]
    icon=folium.Icon(color='orange', prefix='startup')
    start_group.add_child(folium.Marker(coord,popup="Startup "+str(coord),icon=icon))

for star in list(starbucks_2):
    coord = star["location"]["coordinates"][::-1]
    icon=folium.Icon(color='darkgreen', prefix='starbucks')
    starbucks_group.add_child(folium.Marker(coord,popup="Starbucks "+str(coord),icon=icon))
    
for club in list(night_2):
    coord = club["location"]["coordinates"][::-1]
    icon=folium.Icon(color='purple', prefix='Night bar')
    night_group.add_child(folium.Marker(coord,popup="Night bar "+str(coord),icon=icon))

icon=folium.Icon(icon='cloud',color='lightblue', prefix='Airport')
plane_group.add_child(folium.Marker(air_coord,popup="Airport "+str(air_coord),icon=icon))
    
folium.LayerControl(collapsed=True).add_to(m2)
folium.LatLngPopup().add_to(m2)

draw = Draw(export=True)



formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
MousePosition(
    position='topright',
    separator=' | ',
    lng_first=True,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m2)

folium.Circle([42.36022870301706,-71.05980241470984],radius=1000).add_to(m2)

draw.add_to(m2)





In [108]:
def minDist(c1,c2,c3,c4):
    dist = geopy.distance.distance(c1,c2).km
    return f"Distance: {round(dist,2)} Km"


In [111]:
selector_coords = [0,0]
airport_coords = [0,0]
club_coords = [0,0]
startup_coords = [0,0]
starbucks_coords = [0,0]
schools_coords = [0,0]
filtered_coords = [0,0]

distances = ["airport","night clubs","sucesful startup","starbucks","school","old company"]
coords = [airport_coords,club_coords,startup_coords,starbucks_coords,schools_coords,filtered_coords]
for i in range(6):
    dist = minDist(selector[::-1],coords[i])
    print(f"Distance between selected point and the nearest {distances[i]}: {dist}")
    
    
    
    
    
    
    
    

Distance between selected point and the nearest airport: Distance: 0.0 Km
Distance between selected point and the nearest night clubs: Distance: 0.0 Km
Distance between selected point and the nearest sucesful startup: Distance: 0.0 Km
Distance between selected point and the nearest starbucks: Distance: 0.0 Km
Distance between selected point and the nearest school: Distance: 0.0 Km
Distance between selected point and the nearest old company: Distance: 0.0 Km


In [178]:
def nearestPlaces(collection,selector):
    x = collection.find({"location":{"$near":
    {"$geometry":{"type": "Point","coordinates":selector},}}})
    return list(x)

In [180]:
nearestPlaces(start,[42.3656132, -71.0095602])

[{'_id': ObjectId('5dd3fa5727975f68e80f8f35'),
  'Name': 'Happier',
  'Profile URL': 'https://angel.co/company/happier',
  'Signal': '5',
  'Joined': "Jan '13",
  'Location': 'Boston',
  'Market': 'Personal Health',
  'Website': 'https://happier.com/',
  'Employees': '1-10',
  'Stage': 'Seed',
  'Total Raised': '4400000',
  'location': {'type': 'Point',
   'coordinates': [-71.14529259999999, 41.7453265]}},
 {'_id': ObjectId('5dd3fa5727975f68e80f8f0c'),
  'Name': 'Crayon',
  'Profile URL': 'https://angel.co/company/crayon',
  'Signal': '5',
  'Joined': "Feb '15",
  'Location': 'Boston',
  'Market': 'Brand Marketing',
  'Website': 'https://www.crayon.co',
  'Employees': '51-200',
  'Stage': 'Series A',
  'Total Raised': '15850000',
  'location': {'type': 'Point', 'coordinates': [-70.7176946, 41.9241351]}},
 {'_id': ObjectId('5dd3fa5727975f68e80f8f27'),
  'Name': 'ThriveHive',
  'Profile URL': 'https://angel.co/company/thrivehive',
  'Signal': '5',
  'Joined': "Sep '11",
  'Location': 'Bo